#Introduction
This is a notebook using machine learning to output the schedule of suppliers.
First import the data in the form of a csv file. The input of the model will be a date and the output will be a list of probabilities that the supplier will arrive during the day.

# Imports

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import datetime

AssertionError: Duplicate registrations for type 'experimentalOptimizer'

# Load Dataset

In [ ]:
filepath = 'capstone/supplier_schedule.csv'#supplier schedule on local drive
#filepath = '/content/drive/MyDrive/Documents/uni_work/Bangkit2023/capstone/supplier_schedule.csv'
df = pd.read_csv(filepath, dtype = {'supplier':str,'tanggal':str})
df['supplier'] = df['supplier'].str.replace(' ','')



# Data Transformation

Transform the code of suppliers into their tokenized forms using the tokenizer.

In [ ]:
def supplier_tokenizer(supplier):
  """
  Instantiates the tokenizer for the supplier kodes
  Args:
    supplier (list): upper case list of supplier codes
  
  Returns:
    tokenizer (object): an instance of the Tokenizer class containing the dict
  """

  tokenizer = Tokenizer(oov_token='<OOV>')
  tokenizer.fit_on_texts(supplier)

  supplier_index = tokenizer.word_index
  supplier_sequence = tokenizer.texts_to_sequences(supplier)

  return supplier_index, supplier_sequence

supplier_index, supplier_sequence = supplier_tokenizer(df['supplier'])

print(supplier_index)





In [ ]:
#multi_hot encode the suppliers
df['supplier'] = np.array([i[0] for i in supplier_sequence],dtype = np.int64)

multi_hot = pd.get_dummies(df, columns = ['supplier'],prefix = '', prefix_sep = '')
multi_hot = multi_hot.groupby('tanggal').sum()
multi_hot['1'] = [0 for i in range(len(multi_hot['2']))]
multi_hot.insert(0,'1',multi_hot.pop('1'))
multi_hot = multi_hot.reset_index()
multi_hot

In [ ]:
#convert date to epoch
epoch = pd.to_datetime(multi_hot['tanggal'], format = "%Y-%m-%d")
epoch = (epoch - datetime.datetime(1970,1,1))
epoch = epoch.map(lambda x: x.total_seconds())
multi_hot['tanggal'] = epoch


In [ ]:
#sort from latest date
sorted = multi_hot.sort_values('tanggal')#, ascending= False)
sorted = sorted.reset_index()
sorted.pop('index')
sorted

In [ ]:
test = sorted.copy()
test['previous_tanggal'] = test['tanggal'].shift(1)
test

In [ ]:
def create_target_rows(column, tanggal, previous_tanggal):

  returned = [None for i in range(len(column))]
  for i in range(len(column)):
    if current == 0:
      start = i
    current += 1
    if column[i] == 1:
      if current > longest:
        longest_start = start
        end = i
        longest = current
      if current < shortest:
        shortest = current
      current = 0

  return (longest, shortest, longest_start, end)


pos = [str(i) for i in range(2,200)]

for loc in pos:
  print(initiate_first_row(test[loc]))

In [ ]:
test.head()

# Model Architecture

# Analysis